In [8]:
import pandas as pd
from sklearn.svm import SVR
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline

In [2]:
data=pd.read_csv("data.csv")

In [7]:
data

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
0,0.98,514.5,294.0,110.25,7.0,2,0.0,0,15.55,21.33
1,0.98,514.5,294.0,110.25,7.0,3,0.0,0,15.55,21.33
2,0.98,514.5,294.0,110.25,7.0,4,0.0,0,15.55,21.33
3,0.98,514.5,294.0,110.25,7.0,5,0.0,0,15.55,21.33
4,0.90,563.5,318.5,122.50,7.0,2,0.0,0,20.84,28.28
...,...,...,...,...,...,...,...,...,...,...
763,0.64,784.0,343.0,220.50,3.5,5,0.4,5,17.88,21.40
764,0.62,808.5,367.5,220.50,3.5,2,0.4,5,16.54,16.88
765,0.62,808.5,367.5,220.50,3.5,3,0.4,5,16.44,17.11
766,0.62,808.5,367.5,220.50,3.5,4,0.4,5,16.48,16.61


In [3]:
data.describe()

,X1,X2,X3,X4,X5,X6,X7,X8,Y1,Y2
count,768.000000,768.000000,768.000000,768.000000,768.00000,768.000000,768.000000,768.00000,768.000000,768.000000
mean,0.764167,671.708333,318.500000,176.604167,5.25000,3.500000,0.234375,2.81250,22.307201,24.587760
std,0.105777,88.086116,43.626481,45.165950,1.75114,1.118763,0.133221,1.55096,10.090196,9.513306
min,0.620000,514.500000,245.000000,110.250000,3.50000,2.000000,0.000000,0.00000,6.010000,10.900000
25%,0.682500,606.375000,294.000000,140.875000,3.50000,2.750000,0.100000,1.75000,12.992500,15.620000
50%,0.750000,673.750000,318.500000,183.750000,5.25000,3.500000,0.250000,3.00000,18.950000,22.080000
75%,0.830000,741.125000,343.000000,220.500000,7.00000,4.250000,0.400000,4.00000,31.667500,33.132500
max,0.980000,808.500000,416.500000,220.500000,7.00000,5.000000,0.400000,5.00000,43.100000,48.030000


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 10 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X1      768 non-null    float64
 1   X2      768 non-null    float64
 2   X3      768 non-null    float64
 3   X4      768 non-null    float64
 4   X5      768 non-null    float64
 5   X6      768 non-null    int64  
 6   X7      768 non-null    float64
 7   X8      768 non-null    int64  
 8   Y1      768 non-null    float64
 9   Y2      768 non-null    float64
dtypes: float64(8), int64(2)
memory usage: 60.1 KB


In [9]:
Y1=data["Y1"]
Y2=data["Y2"]
X=data.iloc[:,0:8]

In [50]:
pipe1 = Pipeline([('scaler', StandardScaler()), ('svr', SVR(C=10))])
pipe2 = Pipeline([('scaler', StandardScaler()), ('svr', SVR(C=15))])
for _ in range(50):
    X_train,X_test,Y_train1,Y_test1=train_test_split(X,Y1,train_size=0.85)
    Y_test2=Y2[Y_test1.index]
    Y_train2=Y2[Y_train1.index]
    
    pipe1.fit(X_train.values,Y_train1.values)
    pipe2.fit(X_train.values,Y_train2.values)

    print({"Heating Load":mean_absolute_error(pipe1.predict(X_train.values),Y_train1.values),
           "Cooling Load":mean_absolute_error(pipe2.predict(X_train.values),Y_train2.values)})
    
    y1_predict=pipe1.predict(X_test.values)
    y2_predict=pipe2.predict(X_test.values)
    y1_error = mean_absolute_error(y1_predict,Y_test1.values)
    y2_error = mean_absolute_error(y2_predict,Y_test2.values)
    if y1_error < 1.23 and y2_error <1.42:
        break
    print("Test Error",{"Heating Load":y1_error,"Cooling Load":y2_error},"\n")

{'Heating Load': 1.115556366605283, 'Cooling Load': 1.3663605473578007}
Test Error {'Heating Load': 1.2783124164742237, 'Cooling Load': 1.6493643508625557} 

{'Heating Load': 1.1652064122557184, 'Cooling Load': 1.4161382955812158}
Test Error {'Heating Load': 1.1341991132164686, 'Cooling Load': 1.4350329099180876} 

{'Heating Load': 1.1163755437165102, 'Cooling Load': 1.4188039683901004}
Test Error {'Heating Load': 1.3341945299795053, 'Cooling Load': 1.3587585395154074} 

{'Heating Load': 1.0776674429203195, 'Cooling Load': 1.3874749053351572}
Test Error {'Heating Load': 1.532893669080792, 'Cooling Load': 1.7112831712488012} 

{'Heating Load': 1.1660330066437359, 'Cooling Load': 1.4157021634468752}


In [51]:
class Model:
    def __init__(self,model1,model2):
        self.model1=model1
        self.model2=model2
    def predict(self,X):
        return {"Heating Load":self.model1.predict(X)[0],"Cooling Load":self.model2.predict(X)[0]}

In [53]:
model = Model(pipe1,pipe2)
model.predict(X_test.iloc[0].values.reshape(1,-1))

{'Heating Load': 32.58445513876016, 'Cooling Load': 33.43081338270663}

In [54]:
pd.to_pickle(model,"svm_model.pkl")

In [56]:
Y_test2.iloc[0]

33.06